In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import pandas as pd

# Initialize Spark session
spark = SparkSession.builder.appName("ComplexTypesStandalone").getOrCreate()

In [ ]:
# Sample inline dataset simulating sales with nested "items"
sample_data = [
    {
        "email": "alice@example.com",
        "items": [
            {"item_name": "Twin Mattress Firm"},
            {"item_name": "Pillow Soft"}
        ]
    },
    {
        "email": "bob@example.com",
        "items": [
            {"item_name": "Queen Mattress Plush"},
            {"item_name": "Bed Frame"}
        ]
    },
    {
        "email": "charlie@example.com",
        "items": [
            {"item_name": "King Mattress Medium"},
            {"item_name": "Duvet Cover"}
        ]
    }
]

In [ ]:
# Convert to Pandas DataFrame then to Spark DataFrame
df = spark.createDataFrame(pd.json_normalize(sample_data, record_path='items', meta=['email']))
df = df.withColumnRenamed("item_name", "items.item_name")

In [ ]:
# Explode item_name and derive details
exploded_df = df.withColumn("item_name", col("items.item_name"))

# Generate details DataFrame
details_df = (exploded_df
              .select("email", "item_name")
              .withColumn("details", split(col("item_name"), " ")))
details_df.show(truncate=False)

In [ ]:
# Example of string functions
details_df.select(split(details_df.email, '@')[0].alias('email_handle')).show(truncate=False)

In [ ]:
# Filter and process mattresses using collection functions
mattress_df = (details_df
               .filter(array_contains(col("details"), "Mattress"))
               .withColumn("size", element_at(col("details"), 1)))
mattress_df.show(truncate=False)

In [ ]:
# Group sizes by email
size_df = mattress_df.groupBy("email").agg(collect_set("size").alias("size options"))
size_df.show(truncate=False)

In [ ]:
# Demonstrate union and validate row counts
union_count = mattress_df.select("email").union(size_df.select("email")).count()
mattress_count = mattress_df.count()
size_count = size_df.count()
print("Union count matches sum of counts:", union_count == (mattress_count + size_count))

In [ ]:
# Stop Spark session
spark.stop()